In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("fer2013.csv")

In [3]:
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [4]:
data["Usage"].value_counts()

Training       28709
PublicTest      3589
PrivateTest     3589
Name: Usage, dtype: int64

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Dropout, Conv2D,Activation
from tensorflow.keras.layers import AveragePooling2D

In [6]:
data.shape

(35887, 3)

In [7]:
pix_col = data["pixels"]

In [8]:
X_train, y_train, X_test, y_test, X_val, y_val = [],[],[],[],[],[]
for index, row_value in data.iterrows():
    value = row_value["pixels"].split(" ")
    try:
        if "Training" in row_value["Usage"]:
            X_train.append(np.array(value, 'float32'))
            y_train.append(row_value["emotion"])
        elif "PublicTest" in row_value["Usage"]:
            X_test.append(np.array(value, "float32"))
            y_test.append(row_value["emotion"])
        else:
            X_val.append(np.array(value, "float32"))
            y_val.append(row_value["emotion"])
    except Exception as e:
        print(e)

In [9]:
X_train = np.array(X_train)
X_test = np.array(X_test)
X_val = np.array(X_val)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)

In [10]:
X_train

array([[ 70.,  80.,  82., ..., 106., 109.,  82.],
       [151., 150., 147., ..., 193., 183., 184.],
       [231., 212., 156., ...,  88., 110., 152.],
       ...,
       [ 74.,  81.,  87., ..., 188., 187., 187.],
       [222., 227., 203., ..., 136., 136., 134.],
       [195., 199., 205., ...,   6.,  15.,  38.]], dtype=float32)

0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral

In [11]:
img_row = 48
img_col = 48
num_label = 7

from keras.utils import to_categorical, np_utils
y_train = np_utils.to_categorical(y_train, num_classes=num_label)
y_test = np_utils.to_categorical(y_test, num_classes=num_label)
y_val = np_utils.to_categorical(y_val, num_classes=num_label)

Using TensorFlow backend.


In [12]:
X_train = X_train / 255.0
X_test = X_test / 255.0
X_val = X_val / 255.0

In [13]:
X_train = X_train.reshape(X_train.shape[0], img_row, img_col, 1)
X_test = X_test.reshape(X_test.shape[0], img_row, img_col, 1)
X_val =X_val.reshape(X_val.shape[0], img_row, img_col, 1)

In [14]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding="same", input_shape=X_train.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation="relu"))

model.add(Dense(num_label))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

history = model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), shuffle=True)

Epoch 1/20
898/898 [==============================] - 179s 199ms/step - loss: 1.6910 - accuracy: 0.3222 - val_loss: 1.5142 - val_accuracy: 0.4257
Epoch 2/20
898/898 [==============================] - 180s 201ms/step - loss: 1.4486 - accuracy: 0.4433 - val_loss: 1.3516 - val_accuracy: 0.4770
Epoch 3/20
898/898 [==============================] - 179s 199ms/step - loss: 1.3289 - accuracy: 0.4920 - val_loss: 1.2810 - val_accuracy: 0.5138
Epoch 4/20
898/898 [==============================] - 183s 204ms/step - loss: 1.2627 - accuracy: 0.5182 - val_loss: 1.2219 - val_accuracy: 0.5366
Epoch 5/20
898/898 [==============================] - 190s 212ms/step - loss: 1.2083 - accuracy: 0.5405 - val_loss: 1.1798 - val_accuracy: 0.5522
Epoch 6/20
898/898 [==============================] - 187s 208ms/step - loss: 1.1623 - accuracy: 0.5585 - val_loss: 1.1516 - val_accuracy: 0.5631
Epoch 7/20
898/898 [==============================] - 181s 202ms/step - loss: 1.1237 - accuracy: 0.5727 - val_loss: 1.1427 -

In [15]:
scores = model.evaluate(X_test, y_test)

model.save("facecnn.model")

113/113 [==============================] - 5s 46ms/step - loss: 1.2482 - accuracy: 0.5712
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: facecnn.model\assets


In [19]:
import tensorflow as tf
model = tf.keras.models.load_model("facecnn.model")

In [23]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

while True:
    ret, test_img = cap.read()
    if not ret:
        continue
    
    gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray_img, 1.3, 5)
    
    for (x,y,w,h) in faces:
        img = cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray_img[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis=0)
        img_pixels /= 255.0
        
        predictions = model.predict(img_pixels)
        max_index = np.argmax(predictions[0])
        emotions = ("angry", "disgust", "fear", "happy", "sad", "surprise", "neutral")
        predict_emo = emotions[max_index]
        cv2.putText(test_img, predict_emo, (int(x), int(y)), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
    resized_img = cv2.resize(test_img, (1000, 700))
    cv2.imshow("Emotion Detect", resized_img)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

NameError: name 'image' is not defined